In [1]:
import yaml

from odl.target import Target, TargetList
from odl.offset import Stare, StarSkyStar, SkyStar, OffsetPattern, TelescopeOffset, SkyFrame
from odl.block import ObservingBlockList, ScienceBlock, StandardStarBlock
from odl.alignment import BlindAlign, GuiderAlign, MaskAlign

from astropy import units as u

In [2]:
# Define some targets
ngc1333 = Target('NGC1333', rotmode='PA', PA=22.5)
feige110 = Target('Feige110')

In [3]:
print(ngc1333.to_yaml())

Dec: 31.31
DecOffset: null
PA: 22.5
PMDec: 0
PMRA: 0
RA: 52.297
RAOffset: null
comment: null
ddec: 0
dra: 0
epoch: 2021.1096378905663
equinox: 1999.9999085590325
frame: icrs
mag: {}
name: NGC1333
obstime: null
rotmode: PA
wrap: null



In [4]:
print(feige110.to_yaml())

Dec: -5.16560286
DecOffset: null
PA: null
PMDec: 0
PMRA: 0
RA: 349.99333127
RAOffset: null
comment: null
ddec: 0
dra: 0
epoch: 2021.1096378910545
equinox: 1999.9999085590325
frame: icrs
mag: {}
name: Feige110
obstime: null
rotmode: null
wrap: null



The following test cases are meant to be test particles for ODL implementations.  These are meant not to demonstrate a typical observation (though some do), but to challenge a language to describe something which may be somewhat non-standard.

### 1) A KCWI+KCRM Observation

First, observe a single target with two blue exposures on target and a third on an offset sky position between them (in a star-sky-star pattern).  The offset pattern should use a sky position offset from the target by +60 arcseconds in RA and +30 arcseconds in Dec.  The blue exposures should be 1800s long and simultaneously to the blue exposures, the red side should take a pair of shorter exposures (approximately 900s long each).

Second, observe a standard star observation in a single staring observation with both red and blue exposures of 10 seconds.  This will be a staring observation with no repeats.  The science observation above should store the information that this standard star observation is to be used by the DRP for that science observation.

In [5]:
from odl.kcwi import KCWIblueDetectorConfig, KCWIredDetectorConfig, KCWIConfig, mira

In [6]:
# Define my long science exposures
kcwib_1800s = KCWIblueDetectorConfig(exptime=1800, nexp=1, binning='2x2', readoutmode=0, ampmode=9)
kcwir_1800s = KCWIredDetectorConfig(exptime=900, nexp=2, binning='2x2', readoutmode=0, ampmode=9)
kcwir_1800s.match_time(kcwib_1800s.estimate_clock_time())
# Define the short exposures for the standard star
kcwib_10s = KCWIblueDetectorConfig(exptime=10, nexp=1, binning='2x2', readoutmode=0, ampmode=9)
kcwir_10s = KCWIredDetectorConfig(exptime=10, nexp=1, binning='2x2', readoutmode=0, ampmode=9)
# Define my instrument config
med_slicer_b4800_r6500 = KCWIConfig(slicer='medium', bluegrating='BH3', bluecwave=4800,
                                    redgrating='RH3', redcwave=6563)
# Define the offset pattern to use for the science exposures
skyoffset = StarSkyStar(dx=60*u.arcsec, dy=30*u.arcsec, repeat=1)
stare = Stare()
# Define the standard star OB
std = StandardStarBlock(target=feige110, pattern=stare,
                         instconfig=med_slicer_b4800_r6500,
                         detconfig=(kcwib_10s, kcwir_10s),
                         align=BlindAlign(),
                        )
# Define the science OB
sci = ScienceBlock(target=ngc1333, pattern=skyoffset,
                    instconfig=med_slicer_b4800_r6500,
                    detconfig=(kcwib_1800s, kcwir_1800s),
                    align=BlindAlign(),
                    associatedblocks=std,
                   )
OBs = ObservingBlockList([sci, std])
OBs

Target         |Pattern               |InstrumentConfig                             |DetectorConfig                      |AlignmentMethod     
---------------|----------------------|---------------------------------------------|------------------------------------|--------------------
NGC1333        |StarSkyStar (60 30) x1|medium BH3 4800 A                            |(KCWIblue 1800s x1, KCWIred 926s x2)|Blind Align         
Feige110       |Stare x1              |medium BH3 4800 A                            |(KCWIblue 10s x1, KCWIred 10s x1)   |Blind Align         

In [7]:
# Short Version
[OB.to_dict(usenames=True) for OB in OBs]

[{'target': 'NGC1333',
  'pattern': 'StarSkyStar (60 30) x1',
  'instconfig': 'medium BH3 4800 A',
  'detconfig': ['KCWIblue 1800s x1', 'KCWIred 926s x2'],
  'align': 'Blind Align',
  'blocktype': 'Science',
  'associatedblocks': ['Feige110       |Stare x1              |medium BH3 4800 A                            |(KCWIblue 10s x1, KCWIred 10s x1)   |Blind Align         '],
  'guidestar': None,
  'drp_args': None,
  'ql_args': None},
 {'target': 'Feige110',
  'pattern': 'Stare x1',
  'instconfig': 'medium BH3 4800 A',
  'detconfig': ['KCWIblue 10s x1', 'KCWIred 10s x1'],
  'align': 'Blind Align',
  'blocktype': 'StandardStar',
  'associatedblocks': ['None'],
  'guidestar': None,
  'drp_args': None,
  'ql_args': None}]

In [8]:
# Long Version
print(OBs.to_yaml())

- align:
    name: Blind Align
  associatedblocks:
  - 'Feige110       |Stare x1              |medium BH3 4800 A                            |(KCWIblue
    10s x1, KCWIred 10s x1)   |Blind Align         '
  blocktype: Science
  detconfig:
  - ampmode: 9
    binning: 2x2
    dark: false
    detector: blue
    exptime: 1800
    instrument: KCWI
    name: KCWIblue 1800s x1
    nexp: 1
    readoutmode: 0
    window: null
  - ampmode: 9
    binning: 2x2
    dark: false
    detector: red
    exptime: 926.5
    instrument: KCWI
    name: KCWIred 926s x2
    nexp: 2
    readoutmode: 0
    window: null
  drp_args: null
  guidestar: null
  instconfig:
    arclamp: null
    bluecwave: 4800
    bluefilter: KBlue
    bluefocus: null
    bluegrating: BH3
    bluenandsmask: false
    bluepwave: null
    calmirror: Sky
    calobj: Dark
    domeflatlamp: null
    instrument: KCWI
    name: medium BH3 4800 A
    polarizer: Sky
    redcwave: 6563
    redfilter: KRed
    redfocus: null
    redgrating: RH3


### 2) A NIRES SPEC+SCAM Observation

Acquire a faint target (which needs sky subtraction in SCAM) on to the NIRES slit and observe it with 5 repeats of an ABBA sequence with one 300 second SPEC exposure and ten 10 second SCAM exposures at each point in the ABBA sequence.

In [9]:
from odl.nires import NIRESConfig, NIRESSpecDetectorConfig, NIRESScamDetectorConfig, ABBA, mira

In [10]:
# Define the SPEC and SCAM Exposures
spec_300s = NIRESSpecDetectorConfig(exptime=300, readoutmode='MCDS32', coadds=1)
scam_10s = NIRESScamDetectorConfig(exptime=10, readoutmode='CDS', coadds=1, nexp=6)
# Define a NIRES instrument config
nires = NIRESConfig()
# Define the ABBA pattern to use
abba5 = ABBA(offset=10*u.arcsec, repeat=5)
# Define the OB
OB1 = ScienceBlock(target=ngc1333, pattern=abba5, instconfig=nires, detconfig=[spec_300s, scam_10s], align=GuiderAlign(faint=True))
OBs = ObservingBlockList([OB1])
OBs

Target         |Pattern               |InstrumentConfig                             |DetectorConfig                      |AlignmentMethod     
---------------|----------------------|---------------------------------------------|------------------------------------|--------------------
NGC1333        |ABBA (10.00 arcsec) x5|NIRES Instrument Config                      |[NIRES Spec 300s (MCDS32, 1 coadds) x1, NIRES SCAM 10s (CDS, 1 coadds) x6]|Guider Align, faint 

In [11]:
# Short Version
[OB.to_dict(usenames=True) for OB in OBs]

[{'target': 'NGC1333',
  'pattern': 'ABBA (10.00 arcsec) x5',
  'instconfig': 'NIRES Instrument Config',
  'detconfig': ['NIRES Spec 300s (MCDS32, 1 coadds) x1',
   'NIRES SCAM 10s (CDS, 1 coadds) x6'],
  'align': 'Guider Align, faint',
  'blocktype': 'Science',
  'associatedblocks': ['None'],
  'guidestar': None,
  'drp_args': None,
  'ql_args': None}]

In [12]:
# Long Version
print(OBs.to_yaml())

- align:
    faint: true
    name: Guider Align, faint
  associatedblocks:
  - None
  blocktype: Science
  detconfig:
  - coadds: 1
    detector: ''
    exptime: 300
    instrument: NIRES Spec
    name: NIRES Spec 300s (MCDS32, 1 coadds) x1
    nexp: 1
    readoutmode: MCDS32
  - coadds: 1
    detector: ''
    exptime: 10
    instrument: NIRES SCAM
    name: NIRES SCAM 10s (CDS, 1 coadds) x6
    nexp: 6
    readoutmode: CDS
  drp_args: null
  guidestar: null
  instconfig:
    instrument: NIRES
    name: NIRES Instrument Config
  pattern:
    name: ABBA (10.00 arcsec) x5
    offsets:
    - dr: 0.0
      dx: 0.0
      dy: 10.0
      frame: NIRES Slit
      guide: true
      posname: A
      relative: false
    - dr: 0.0
      dx: 0.0
      dy: -10.0
      frame: NIRES Slit
      guide: true
      posname: B
      relative: false
    - dr: 0.0
      dx: 0.0
      dy: -10.0
      frame: NIRES Slit
      guide: true
      posname: B
      relative: false
    - dr: 0.0
      dx: 0.0
      dy

### 3) A MOSFIRE MOS Mask

Observe a MOS mask using MOSFIRE in spectroscopy mode in the Y, J, H, and K filters.  Use the reccommended exposure time of either 120 or 180 seconds (depending on filter) with MCDS16 readout mode.  For the filters with 180 second exposures, use 5 repeats of an ABBA sequence, for the filters with 120 second exposures use 8 repeats.  The ABBA sequence should have a dither magnitude of 1.25 arcseconds.

In [13]:
from odl.mosfire import MOSFIREConfig, MOSFIREDetectorConfig, long2pos, ABBA, mira, MOSFIREMaskAlign, MOSFIREBrightStarAlign

#### MOSFIRE Mask Note

In this example, the MOSFIRE mask is simply a string with the name of the mask.  In an actual implementaion, it would have to be either a an xml file or a reference to a database entry containing the mask design (which would contain the same info as the xml file).

In [14]:
# Define my long science exposures
mosfire_180s = MOSFIREDetectorConfig(exptime=180, readoutmode='MCDS16', coadds=1)
mosfire_120s = MOSFIREDetectorConfig(exptime=120, readoutmode='MCDS16', coadds=1)
# Define my instrument config
mosfire_Y = MOSFIREConfig(mode='spectroscopy', filter='Y', mask='my_MOS_mask')
mosfire_J = MOSFIREConfig(mode='spectroscopy', filter='J', mask='my_MOS_mask')
mosfire_H = MOSFIREConfig(mode='spectroscopy', filter='H', mask='my_MOS_mask')
mosfire_K = MOSFIREConfig(mode='spectroscopy', filter='K', mask='my_MOS_mask')
# Define the offset patterns to use for the science exposures
abba5 = ABBA(offset=1.25*u.arcsec, repeat=5)
abba8 = ABBA(offset=1.25*u.arcsec, repeat=8)
# Define the science OB
OBs = ObservingBlockList([ScienceBlock(target=ngc1333, pattern=abba5, instconfig=mosfire_Y, detconfig=mosfire_180s, align=MOSFIREMaskAlign()),
                          ScienceBlock(target=ngc1333, pattern=abba8, instconfig=mosfire_J, detconfig=mosfire_120s, align=None),
                          ScienceBlock(target=ngc1333, pattern=abba8, instconfig=mosfire_H, detconfig=mosfire_120s, align=None),
                          ScienceBlock(target=ngc1333, pattern=abba5, instconfig=mosfire_K, detconfig=mosfire_180s, align=None),
                         ])
OBs

Target         |Pattern               |InstrumentConfig                             |DetectorConfig                      |AlignmentMethod     
---------------|----------------------|---------------------------------------------|------------------------------------|--------------------
NGC1333        |ABBA (1.25 arcsec) x5 |my_MOS_mask Y-spectroscopy                   |[MOSFIRE 180s (MCDS16, 1 coadds) x1]|Mask Align (MOSFIRE 7s (CDS, 3 coadds) x1) take sky
NGC1333        |ABBA (1.25 arcsec) x8 |my_MOS_mask J-spectroscopy                   |[MOSFIRE 120s (MCDS16, 1 coadds) x1]|None                
NGC1333        |ABBA (1.25 arcsec) x8 |my_MOS_mask H-spectroscopy                   |[MOSFIRE 120s (MCDS16, 1 coadds) x1]|None                
NGC1333        |ABBA (1.25 arcsec) x5 |my_MOS_mask K-spectroscopy                   |[MOSFIRE 180s (MCDS16, 1 coadds) x1]|None                

In [15]:
# Short Version
[OB.to_dict(usenames=True) for OB in OBs]

[{'target': 'NGC1333',
  'pattern': 'ABBA (1.25 arcsec) x5',
  'instconfig': 'my_MOS_mask Y-spectroscopy',
  'detconfig': ['MOSFIRE 180s (MCDS16, 1 coadds) x1'],
  'align': 'Mask Align (MOSFIRE 7s (CDS, 3 coadds) x1) take sky',
  'blocktype': 'Science',
  'associatedblocks': ['None'],
  'guidestar': None,
  'drp_args': None,
  'ql_args': None},
 {'target': 'NGC1333',
  'pattern': 'ABBA (1.25 arcsec) x8',
  'instconfig': 'my_MOS_mask J-spectroscopy',
  'detconfig': ['MOSFIRE 120s (MCDS16, 1 coadds) x1'],
  'align': 'None',
  'blocktype': 'Science',
  'associatedblocks': ['None'],
  'guidestar': None,
  'drp_args': None,
  'ql_args': None},
 {'target': 'NGC1333',
  'pattern': 'ABBA (1.25 arcsec) x8',
  'instconfig': 'my_MOS_mask H-spectroscopy',
  'detconfig': ['MOSFIRE 120s (MCDS16, 1 coadds) x1'],
  'align': 'None',
  'blocktype': 'Science',
  'associatedblocks': ['None'],
  'guidestar': None,
  'drp_args': None,
  'ql_args': None},
 {'target': 'NGC1333',
  'pattern': 'ABBA (1.25 arcse

In [16]:
# Long Version
print(OBs.to_yaml())

- align:
    detconfig:
      coadds: 3
      detector: ''
      exptime: 7
      instrument: MOSFIRE
      name: MOSFIRE 7s (CDS, 3 coadds) x1
      nexp: 1
      readoutmode: CDS
    filter: J
    name: Mask Align (MOSFIRE 7s (CDS, 3 coadds) x1) take sky
    takesky: true
  associatedblocks:
  - None
  blocktype: Science
  detconfig:
  - coadds: 1
    detector: ''
    exptime: 180
    instrument: MOSFIRE
    name: MOSFIRE 180s (MCDS16, 1 coadds) x1
    nexp: 1
    readoutmode: MCDS16
  drp_args: null
  guidestar: null
  instconfig:
    arclamp: null
    domeflatlamp: null
    filter: Y
    instrument: MOSFIRE
    mask: my_MOS_mask
    mode: spectroscopy
    name: my_MOS_mask Y-spectroscopy
  pattern:
    name: ABBA (1.25 arcsec) x5
    offsets:
    - dr: 0.0
      dx: 0.0
      dy: 1.25
      frame: MOSFIRE Slit
      guide: true
      posname: A
      relative: false
    - dr: 0.0
      dx: 0.0
      dy: -1.25
      frame: MOSFIRE Slit
      guide: true
      posname: B
      relati

### 4) A MOSFIRE Single Object

Observe a single object using MOSFIRE in spectroscopy mode in the Y, J, H, and K filters.  Use the reccommended exposure time of either 120 or 180 seconds (depending on filter) with MCDS16 readout mode.  Use the longslit for Y and J filters with an ABBA pattern and 2 repeats, but use long2pos mask and a long2pos pattern for the H and K observations.

In [17]:
# Define my long science exposures
mosfire_180s = MOSFIREDetectorConfig(exptime=180, readoutmode='MCDS16', coadds=1)
mosfire_120s = MOSFIREDetectorConfig(exptime=120, readoutmode='MCDS16', coadds=1)
# Define my instrument config
singleobj_Y = MOSFIREConfig(mode='spectroscopy', filter='Y', mask='longslit_3x0.7')
singleobj_J = MOSFIREConfig(mode='spectroscopy', filter='J', mask='longslit_3x0.7')
singleobj_H = MOSFIREConfig(mode='spectroscopy', filter='H', mask='long2pos')
singleobj_K = MOSFIREConfig(mode='spectroscopy', filter='K', mask='long2pos')
# Define the offset patterns to use for the science exposures
abba2 = ABBA(offset=1.25*u.arcsec, repeat=2)
long2pos_pattern = long2pos(repeat=2)
# Define the science OB
OBs = ObservingBlockList([ScienceBlock(target=ngc1333, detconfig=mosfire_180s, pattern=abba2, instconfig=singleobj_Y, align=MOSFIREMaskAlign()),
                          ScienceBlock(target=ngc1333, detconfig=mosfire_120s, pattern=abba2, instconfig=singleobj_J, align=None),
                          ScienceBlock(target=ngc1333, detconfig=mosfire_120s, pattern=long2pos_pattern, instconfig=singleobj_H, align=None),
                          ScienceBlock(target=ngc1333, detconfig=mosfire_180s, pattern=long2pos_pattern, instconfig=singleobj_K, align=None),
                         ])
OBs

Target         |Pattern               |InstrumentConfig                             |DetectorConfig                      |AlignmentMethod     
---------------|----------------------|---------------------------------------------|------------------------------------|--------------------
NGC1333        |ABBA (1.25 arcsec) x2 |longslit_3x0.7 Y-spectroscopy                |[MOSFIRE 180s (MCDS16, 1 coadds) x1]|Mask Align (MOSFIRE 7s (CDS, 3 coadds) x1) take sky
NGC1333        |ABBA (1.25 arcsec) x2 |longslit_3x0.7 J-spectroscopy                |[MOSFIRE 120s (MCDS16, 1 coadds) x1]|None                
NGC1333        |long2pos x2           |long2pos H-spectroscopy                      |[MOSFIRE 120s (MCDS16, 1 coadds) x1]|None                
NGC1333        |long2pos x2           |long2pos K-spectroscopy                      |[MOSFIRE 180s (MCDS16, 1 coadds) x1]|None                

In [18]:
# Short Version
[OB.to_dict(usenames=True) for OB in OBs]

[{'target': 'NGC1333',
  'pattern': 'ABBA (1.25 arcsec) x2',
  'instconfig': 'longslit_3x0.7 Y-spectroscopy',
  'detconfig': ['MOSFIRE 180s (MCDS16, 1 coadds) x1'],
  'align': 'Mask Align (MOSFIRE 7s (CDS, 3 coadds) x1) take sky',
  'blocktype': 'Science',
  'associatedblocks': ['None'],
  'guidestar': None,
  'drp_args': None,
  'ql_args': None},
 {'target': 'NGC1333',
  'pattern': 'ABBA (1.25 arcsec) x2',
  'instconfig': 'longslit_3x0.7 J-spectroscopy',
  'detconfig': ['MOSFIRE 120s (MCDS16, 1 coadds) x1'],
  'align': 'None',
  'blocktype': 'Science',
  'associatedblocks': ['None'],
  'guidestar': None,
  'drp_args': None,
  'ql_args': None},
 {'target': 'NGC1333',
  'pattern': 'long2pos x2',
  'instconfig': 'long2pos H-spectroscopy',
  'detconfig': ['MOSFIRE 120s (MCDS16, 1 coadds) x1'],
  'align': 'None',
  'blocktype': 'Science',
  'associatedblocks': ['None'],
  'guidestar': None,
  'drp_args': None,
  'ql_args': None},
 {'target': 'NGC1333',
  'pattern': 'long2pos x2',
  'instco

In [19]:
# Long Version
print(OBs.to_yaml())

- align:
    detconfig:
      coadds: 3
      detector: ''
      exptime: 7
      instrument: MOSFIRE
      name: MOSFIRE 7s (CDS, 3 coadds) x1
      nexp: 1
      readoutmode: CDS
    filter: J
    name: Mask Align (MOSFIRE 7s (CDS, 3 coadds) x1) take sky
    takesky: true
  associatedblocks:
  - None
  blocktype: Science
  detconfig:
  - coadds: 1
    detector: ''
    exptime: 180
    instrument: MOSFIRE
    name: MOSFIRE 180s (MCDS16, 1 coadds) x1
    nexp: 1
    readoutmode: MCDS16
  drp_args: null
  guidestar: null
  instconfig:
    arclamp: null
    domeflatlamp: null
    filter: Y
    instrument: MOSFIRE
    mask: longslit_3x0.7
    mode: spectroscopy
    name: longslit_3x0.7 Y-spectroscopy
  pattern:
    name: ABBA (1.25 arcsec) x2
    offsets:
    - dr: 0.0
      dx: 0.0
      dy: 1.25
      frame: MOSFIRE Slit
      guide: true
      posname: A
      relative: false
    - dr: 0.0
      dx: 0.0
      dy: -1.25
      frame: MOSFIRE Slit
      guide: true
      posname: B
      

### 5) An Unusual MOSFIRE Observation

First, acquire a target using an offset star and a MOS mask alignment process with a MOSFIRE long slit.

Second, after offsetting to target, image the target in J-band through the alignment mask.  Use 9 coadds of 10 seconds each.

Finally, take Y and J longslit spectra of the target using an ABBA pattern with 3 repeats and standard MOSFIRE exposure parameters (e.g. 180s and 120s exposure times).

In [20]:
# Define my science exposures
mosfire_30sx3 = MOSFIREDetectorConfig(exptime=30, readoutmode='CDS', coadds=3)
mosfire_10sx9 = MOSFIREDetectorConfig(exptime=10, readoutmode='CDS', coadds=9)
mosfire_180s = MOSFIREDetectorConfig(exptime=180, readoutmode='MCDS16', coadds=1)
mosfire_120s = MOSFIREDetectorConfig(exptime=120, readoutmode='MCDS16', coadds=1)
# Define my instrument config
#   Build explicit align version of the mask so it can be sent as unique science mask
singleobj_J_align = MOSFIREConfig(mode='imaging', filter='J', mask='longslit_46x0.7', alignmask=True)
#   Build normal spectroscopy masks
singleobj_J = MOSFIREConfig(mode='spectroscopy', filter='J', mask='longslit_46x0.7')
singleobj_Y = MOSFIREConfig(mode='spectroscopy', filter='Y', mask='longslit_46x0.7')
# Define the offset patterns to use for the science exposures
skystar = SkyStar(dx=10*u.arcsec, dy=10*u.arcsec)
abba3 = ABBA(offset=1.25*u.arcsec, repeat=3)
# Define the science OB
OBs = ObservingBlockList([ScienceBlock(target=ngc1333, pattern=skystar, instconfig=singleobj_J_align, detconfig=mosfire_10sx9, align=MOSFIREMaskAlign()),
                          ScienceBlock(target=ngc1333, pattern=abba3, instconfig=singleobj_J, detconfig=mosfire_120s, align=None),
                          ScienceBlock(target=ngc1333, pattern=abba3, instconfig=singleobj_Y, detconfig=mosfire_180s, align=None),
                         ])
OBs

Target         |Pattern               |InstrumentConfig                             |DetectorConfig                      |AlignmentMethod     
---------------|----------------------|---------------------------------------------|------------------------------------|--------------------
NGC1333        |SkyStar (10 10) x1    |longslit_46x0.7-align J-imaging              |[MOSFIRE 10s (CDS, 9 coadds) x1]    |Mask Align (MOSFIRE 7s (CDS, 3 coadds) x1) take sky
NGC1333        |ABBA (1.25 arcsec) x3 |longslit_46x0.7 J-spectroscopy               |[MOSFIRE 120s (MCDS16, 1 coadds) x1]|None                
NGC1333        |ABBA (1.25 arcsec) x3 |longslit_46x0.7 Y-spectroscopy               |[MOSFIRE 180s (MCDS16, 1 coadds) x1]|None                

In [21]:
# Short Version
[OB.to_dict(usenames=True) for OB in OBs]

[{'target': 'NGC1333',
  'pattern': 'SkyStar (10 10) x1',
  'instconfig': 'longslit_46x0.7-align J-imaging',
  'detconfig': ['MOSFIRE 10s (CDS, 9 coadds) x1'],
  'align': 'Mask Align (MOSFIRE 7s (CDS, 3 coadds) x1) take sky',
  'blocktype': 'Science',
  'associatedblocks': ['None'],
  'guidestar': None,
  'drp_args': None,
  'ql_args': None},
 {'target': 'NGC1333',
  'pattern': 'ABBA (1.25 arcsec) x3',
  'instconfig': 'longslit_46x0.7 J-spectroscopy',
  'detconfig': ['MOSFIRE 120s (MCDS16, 1 coadds) x1'],
  'align': 'None',
  'blocktype': 'Science',
  'associatedblocks': ['None'],
  'guidestar': None,
  'drp_args': None,
  'ql_args': None},
 {'target': 'NGC1333',
  'pattern': 'ABBA (1.25 arcsec) x3',
  'instconfig': 'longslit_46x0.7 Y-spectroscopy',
  'detconfig': ['MOSFIRE 180s (MCDS16, 1 coadds) x1'],
  'align': 'None',
  'blocktype': 'Science',
  'associatedblocks': ['None'],
  'guidestar': None,
  'drp_args': None,
  'ql_args': None}]

In [22]:
# Long Version
print(OBs.to_yaml())

- align:
    detconfig:
      coadds: 3
      detector: ''
      exptime: 7
      instrument: MOSFIRE
      name: MOSFIRE 7s (CDS, 3 coadds) x1
      nexp: 1
      readoutmode: CDS
    filter: J
    name: Mask Align (MOSFIRE 7s (CDS, 3 coadds) x1) take sky
    takesky: true
  associatedblocks:
  - None
  blocktype: Science
  detconfig:
  - coadds: 9
    detector: ''
    exptime: 10
    instrument: MOSFIRE
    name: MOSFIRE 10s (CDS, 9 coadds) x1
    nexp: 1
    readoutmode: CDS
  drp_args: null
  guidestar: null
  instconfig:
    arclamp: null
    domeflatlamp: null
    filter: J
    instrument: MOSFIRE
    mask: longslit_46x0.7
    mode: imaging
    name: longslit_46x0.7-align J-imaging
  pattern:
    name: SkyStar (10 10) x1
    offsets:
    - dr: 0.0
      dx: 10.0
      dy: 10.0
      frame: SkyFrame
      guide: false
      posname: sky
      relative: false
    - dr: 0.0
      dx: 0.0
      dy: 0.0
      frame: SkyFrame
      guide: true
      posname: star
      relative: false


### 6) An Unusual MOSFIRE Observation: Alternate Observing Strategy

First, acquire a target using an offset star and a MOS mask alignment process with a MOSFIRE long slit.

Second, after offsetting to target, image the target in J-band through the alignment mask.  Use 9 coadds of 10 seconds each.

Third, instead of changing to the “science” long slit mask, take the Y and J spectra by offsetting the target away from the central alignment box.

In [23]:
# Define my science exposures
mosfire_30sx3 = MOSFIREDetectorConfig(exptime=30, readoutmode='CDS', coadds=3)
mosfire_10sx9 = MOSFIREDetectorConfig(exptime=10, readoutmode='CDS', coadds=9)
mosfire_180s = MOSFIREDetectorConfig(exptime=180, readoutmode='MCDS16', coadds=1)
mosfire_120s = MOSFIREDetectorConfig(exptime=120, readoutmode='MCDS16', coadds=1)
# Define my instrument config
#   Build explicit align version of the mask so it can be sent as unique science mask
singleobj_J_align = MOSFIREConfig(mode='imaging', filter='J', mask='longslit_46x0.7', alignmask=True)
singleobj_J_spec = MOSFIREConfig(mode='spectroscopy', filter='J', mask='longslit_46x0.7', alignmask=True)
singleobj_Y_spec = MOSFIREConfig(mode='spectroscopy', filter='Y', mask='longslit_46x0.7', alignmask=True)
# Define the offset patterns to use for the science exposures
skystar = SkyStar(dx=10*u.arcsec, dy=10*u.arcsec)
abba_wide3 = ABBA(offset=7*u.arcsec, repeat=3)
# Define the science OB
OBs = ObservingBlockList([ScienceBlock(target=ngc1333, pattern=skystar, instconfig=singleobj_J_align, detconfig=mosfire_10sx9, align=MOSFIREMaskAlign()),
                          ScienceBlock(target=ngc1333, pattern=abba_wide3, instconfig=singleobj_J_spec, detconfig=mosfire_120s, align=None),
                          ScienceBlock(target=ngc1333, pattern=abba_wide3, instconfig=singleobj_Y_spec, detconfig=mosfire_180s, align=None),
                         ])
OBs

Target         |Pattern               |InstrumentConfig                             |DetectorConfig                      |AlignmentMethod     
---------------|----------------------|---------------------------------------------|------------------------------------|--------------------
NGC1333        |SkyStar (10 10) x1    |longslit_46x0.7-align J-imaging              |[MOSFIRE 10s (CDS, 9 coadds) x1]    |Mask Align (MOSFIRE 7s (CDS, 3 coadds) x1) take sky
NGC1333        |ABBA (7.00 arcsec) x3 |longslit_46x0.7-align J-spectroscopy         |[MOSFIRE 120s (MCDS16, 1 coadds) x1]|None                
NGC1333        |ABBA (7.00 arcsec) x3 |longslit_46x0.7-align Y-spectroscopy         |[MOSFIRE 180s (MCDS16, 1 coadds) x1]|None                

In [24]:
# Short Version
[OB.to_dict(usenames=True) for OB in OBs]

[{'target': 'NGC1333',
  'pattern': 'SkyStar (10 10) x1',
  'instconfig': 'longslit_46x0.7-align J-imaging',
  'detconfig': ['MOSFIRE 10s (CDS, 9 coadds) x1'],
  'align': 'Mask Align (MOSFIRE 7s (CDS, 3 coadds) x1) take sky',
  'blocktype': 'Science',
  'associatedblocks': ['None'],
  'guidestar': None,
  'drp_args': None,
  'ql_args': None},
 {'target': 'NGC1333',
  'pattern': 'ABBA (7.00 arcsec) x3',
  'instconfig': 'longslit_46x0.7-align J-spectroscopy',
  'detconfig': ['MOSFIRE 120s (MCDS16, 1 coadds) x1'],
  'align': 'None',
  'blocktype': 'Science',
  'associatedblocks': ['None'],
  'guidestar': None,
  'drp_args': None,
  'ql_args': None},
 {'target': 'NGC1333',
  'pattern': 'ABBA (7.00 arcsec) x3',
  'instconfig': 'longslit_46x0.7-align Y-spectroscopy',
  'detconfig': ['MOSFIRE 180s (MCDS16, 1 coadds) x1'],
  'align': 'None',
  'blocktype': 'Science',
  'associatedblocks': ['None'],
  'guidestar': None,
  'drp_args': None,
  'ql_args': None}]

In [25]:
# Long Version
print(OBs.to_yaml())

- align:
    detconfig:
      coadds: 3
      detector: ''
      exptime: 7
      instrument: MOSFIRE
      name: MOSFIRE 7s (CDS, 3 coadds) x1
      nexp: 1
      readoutmode: CDS
    filter: J
    name: Mask Align (MOSFIRE 7s (CDS, 3 coadds) x1) take sky
    takesky: true
  associatedblocks:
  - None
  blocktype: Science
  detconfig:
  - coadds: 9
    detector: ''
    exptime: 10
    instrument: MOSFIRE
    name: MOSFIRE 10s (CDS, 9 coadds) x1
    nexp: 1
    readoutmode: CDS
  drp_args: null
  guidestar: null
  instconfig:
    arclamp: null
    domeflatlamp: null
    filter: J
    instrument: MOSFIRE
    mask: longslit_46x0.7
    mode: imaging
    name: longslit_46x0.7-align J-imaging
  pattern:
    name: SkyStar (10 10) x1
    offsets:
    - dr: 0.0
      dx: 10.0
      dy: 10.0
      frame: SkyFrame
      guide: false
      posname: sky
      relative: false
    - dr: 0.0
      dx: 0.0
      dy: 0.0
      frame: SkyFrame
      guide: true
      posname: star
      relative: false


### 7) An unusual single object acquisition with LRIS

First, acquire and perform a MOS observation with LRIS.

Second, using the same MOS mask, put a standard star on one of the slits and observe it there instead of switching to long slit.

The motivation for this was to save reconfig time with LRIS and to capture the standard star spectrum with a wavelength coverage that was more useful for the science than would have been obtained with a normal long slit.  The observers wanted to get the red end of the spectrum which would have gone off the edge of the detector for a normal long slit.  This matched the wavelength coverage of the highest priority slit in their mask.

#### Note

Since this version of ODL has a backend, I did not do this example for LRIS, but for MOSFIRE as I do not have the components programmed for LRIS.  The principles are the same.

#### Explaination

This is obviously a challenging one to describe.  Without building a separate custom alignment tool, here's one way to make this happen with human input along the way.  This uses a "dummy" OB with no observations in order to get the sequencing right.

1) Observe the normal science OB as usual
2) Observe the dummy OB:
    a) Slew to the standard star, but using a custom `Target` which has offsets which places the star in one of the alignment boxes in the mask.  Observers would have to pre-calculate the offsets based on the mask design or based on a mask image from the afternoon.
    b) Since this is a blind alignment, nothing happens for alignment step.
    c) Since the detector config is None, no science exposures are taken.
3) Observe the standard star OB:
    a) The target is `None`, so no slew is made.
    b) Align the target in the alignment box (where it was placed by the previous OB) using the normal process for bright alignment stars.
    c) Observe the target using a special offset pattern which has a single position which is offset from base.  The offsets in that pattern would have to be calculated ahead of time by the observers to move the star from the alignment box to the slit of choice.


In [26]:
# Define my long science exposures
mosfire_180s = MOSFIREDetectorConfig(exptime=180, readoutmode='MCDS16', coadds=1)
mosfire_10s = MOSFIREDetectorConfig(exptime=10, readoutmode='CDS', coadds=1)
# Define my instrument config
mosfire_Y = MOSFIREConfig(mode='spectroscopy', filter='Y', mask='my_MOS_mask')
# Define the offset patterns to use for the science exposures
abba5 = ABBA(offset=1.25*u.arcsec, repeat=5)

# Define the science OBs
sci = ScienceBlock(target=ngc1333, pattern=abba5, instconfig=mosfire_Y,
                   detconfig=mosfire_180s, align=MOSFIREMaskAlign(),
                   associatedblocks=std)

# Define the Special Standard Star OB Components
feige110_offset = feige110 = Target('Feige110',
                                   RAOffset=22.3,
                                   DecOffset=18.5,
                                  )

dummy = StandardStarBlock(target=feige110_offset,
                          pattern=Stare(),
                          instconfig=mosfire_Y,
                          detconfig=None,
                          align=BlindAlign(),
                          )
custom_pattern = OffsetPattern([TelescopeOffset(dx=-35.1*u.arcsec,
                                                dy=-29.9*u.arcsec,
                                                posname='on_slit',
                                                frame=SkyFrame())],
                               name='Special', repeat=1)
std = StandardStarBlock(target=None,
                        pattern=custom_pattern,
                        instconfig=mosfire_Y,
                        detconfig=mosfire_10s,
                        align=MOSFIREBrightStarAlign(),
                        )

OBs = ObservingBlockList([sci,dummy,std])
OBs

Target         |Pattern               |InstrumentConfig                             |DetectorConfig                      |AlignmentMethod     
---------------|----------------------|---------------------------------------------|------------------------------------|--------------------
NGC1333        |ABBA (1.25 arcsec) x5 |my_MOS_mask Y-spectroscopy                   |[MOSFIRE 180s (MCDS16, 1 coadds) x1]|Mask Align (MOSFIRE 7s (CDS, 3 coadds) x1) take sky
Feige110       |Stare x1              |my_MOS_mask Y-spectroscopy                   |[None]                              |Blind Align         
None           |Special x1            |my_MOS_mask Y-spectroscopy                   |[MOSFIRE 10s (CDS, 1 coadds) x1]    |Mask Align (MOSFIRE 2s (CDS, 5 coadds) x1)

In [27]:
print(OBs.to_yaml())

- align:
    detconfig:
      coadds: 3
      detector: ''
      exptime: 7
      instrument: MOSFIRE
      name: MOSFIRE 7s (CDS, 3 coadds) x1
      nexp: 1
      readoutmode: CDS
    filter: J
    name: Mask Align (MOSFIRE 7s (CDS, 3 coadds) x1) take sky
    takesky: true
  associatedblocks:
  - 'Feige110       |Stare x1              |medium BH3 4800 A                            |(KCWIblue
    10s x1, KCWIred 10s x1)   |Blind Align         '
  blocktype: Science
  detconfig:
  - coadds: 1
    detector: ''
    exptime: 180
    instrument: MOSFIRE
    name: MOSFIRE 180s (MCDS16, 1 coadds) x1
    nexp: 1
    readoutmode: MCDS16
  drp_args: null
  guidestar: null
  instconfig:
    arclamp: null
    domeflatlamp: null
    filter: Y
    instrument: MOSFIRE
    mask: my_MOS_mask
    mode: spectroscopy
    name: my_MOS_mask Y-spectroscopy
  pattern:
    name: ABBA (1.25 arcsec) x5
    offsets:
    - dr: 0.0
      dx: 0.0
      dy: 1.25
      frame: MOSFIRE Slit
      guide: true
      posnam